In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model="gpt-4o",
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key,
)


In [2]:
def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:  # type: ignore[type-arg]
    import os
    import time

    import requests
    from bs4 import BeautifulSoup
    from dotenv import load_dotenv

    load_dotenv()

    serp_api_key = os.getenv("SERPAPI_KEY")

    if not serp_api_key:
        raise ValueError("SerpAPI key not found in environment variables")

    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "num": num_results,
        "api_key": serp_api_key,
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(response.json())
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get("organic_results", [])

    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            words = text.split()
            content = ""
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += " " + word
            return content.strip()
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""

    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append(
            {"title": item["title"], "link": item["link"], "snippet": item.get("snippet", ""), "body": body}
        )
        time.sleep(1)  # Be respectful to the servers

    print(enriched_results)    

    return enriched_results


In [3]:
def arxiv_search(query: str, max_results: int = 2) -> list:  # type: ignore[type-arg]
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

    results = []
    for paper in client.results(search):
        results.append(
            {
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "published": paper.published.strftime("%Y-%m-%d"),
                "abstract": paper.summary,
                "pdf_url": paper.pdf_url,
            }
        )


    print(results)
    return results

In [4]:
google_search_tool = FunctionTool(
    google_search, description="Search Google for information, returns results with a snippet and body content"
)
arxiv_search_tool = FunctionTool(
    arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts"
)

In [5]:
google_search_agent = AssistantAgent(
    name="Google_Search_Agent",
    tools=[google_search_tool],
    model_client=azure_openai_chat_completion_client,
    description="An agent that can search Google for information, returns results with a snippet and body content",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client=azure_openai_chat_completion_client,
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts",
    system_message="You are a helpful AI assistant. Solve tasks using your tools. Specifically, you can take into consideration the user's request and craft a search query that is most likely to return relevant academi papers.",
)


report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=azure_openai_chat_completion_client,
    description="Generate a report based on a given topic",
    system_message="You are a helpful assistant. Your task is to synthesize data extracted into a high quality literature review including CORRECT references. You MUST write a final report that is formatted as a literature review with CORRECT references.  Your response should end with the word 'TERMINATE'",
)

In [6]:
termination = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat(
    participants=[google_search_agent, arxiv_search_agent, report_agent], termination_condition=termination
)

In [7]:
await Console(
    team.run_stream(
        task="멀티 에이전트 AI 시스템 구축을 위한한 논문 리뷰 리포트 작성",
        # task="최신 RAG 기술에 대한 동향 및 논문 리뷰",
    )
)

---------- TextMessage (user) ----------
멀티 에이전트 AI 시스템 구축을 위한한 논문 리뷰 리포트 작성
---------- ToolCallRequestEvent (Google_Search_Agent) ----------
[FunctionCall(id='call_nfZ4pzd8c6IBIVmK7ZxLWO4C', arguments='{"query":"멀티 에이전트 AI 시스템 구축 논문","num_results":2}', name='google_search')]


/Users/andy/works/ai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/agents/_assistant_agent.py:981: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


[{'title': '멀티에이전트 강화학습 및 베이지안 의사결정 기반 AI 전투 ...', 'link': 'https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART003155966', 'snippet': '본 논문은 불확실성 하에서 다중 에이전트 강화학습(MARL)과 베이지안 의사결정 프레임워크를 적용한 인공지능(AI) 기반 전투 의사결정 지원 시스템의 성능을 분석한다. 5 ...', 'body': '멀티에이전트 강화학습 및 베이지안 의사결정 기반 AI 전투 의사결정 지원 시스템의 성능 분석 본문 바로가기 허위 인용 신고 오류신고 로그인 Sitemap English KCI 소개 논문 검색 학술지 검색 기관 정보 검색 인용 정보 검색 통계 정보 정보마당 논문유사도검사 분석정보서비스 통합검색 논문 학술지 기관 학술대회 다국어입력 검색 HOME > 논문 > 논문 상세 KCI 등재 멀티에이전트 강화학습 및 베이지안 의사결정 기반 AI 전투 의사결정 지원 시스템의 성능 분석 Performance Analysis of AI-based Combat Decision Support System using Multi-Agent Reinforcement Learning and Bayesian Decision-Making 한국정보기술학회논문지 이 학술지 인용지수 조회 이 학술지 논문 검색 약어 : JKIIT 2024, vol.22, no.12, pp. 65-74 (10 pages) DOI :'}, {'title': '멀티 에이전트 시스템의 효율 향상을 위한 연구 동향 분석', 'link': 'https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11145140', 'snippet': '에이전트기반 모델링. Javad Khazaii , 윤린. 설비저널. 2016 .05 · AI Bots를 위한 멀티에이전트 

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 24, 3, 431937, tzinfo=datetime.timezone.utc), content='멀티 에이전트 AI 시스템 구축을 위한한 논문 리뷰 리포트 작성', type='TextMessage'), ToolCallRequestEvent(source='Google_Search_Agent', models_usage=RequestUsage(prompt_tokens=128, completion_tokens=30), metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 24, 4, 442756, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_nfZ4pzd8c6IBIVmK7ZxLWO4C', arguments='{"query":"멀티 에이전트 AI 시스템 구축 논문","num_results":2}', name='google_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Google_Search_Agent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 20, 7, 24, 11, 264536, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content="[{'title': '멀티에이전트 강화학습 및 베이지안 의사결정 기반 AI 전투 ...', 'link': 'https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSe